In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import timeit
import datetime
import time
import pprint
import itertools
import pickle
import sklearn
import dask
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
import fastparquet
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import h5py
import collections
import dask
import dask.dataframe as dd
import dask.array as da

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
npc = fastparquet.ParquetFile("DataStore/2018-10/mutual_npc_1027.parq").to_pandas()
mdc = fastparquet.ParquetFile("DataStore/2018-07-P2/mutual_cited_0716.parq").to_pandas()

# Only NPC from differing inventors
npc = npc.loc[(npc["common_pat_inv"] == False)]
# Rename column
npc = npc.rename(columns={"direct_cite": "op_cites_tp"})
print(len(npc))

def get_year_group_10(x):
    if x in range(1975,1985):
        yg = "1975-85"
    elif x in range(1985,1995):
        yg = "1985-95"
    elif x in range(1995, 2005):
        yg = "1995-05"
    elif x in range(2005,2015):
        yg = "2005-15"
    else:
        yg = np.nan
    return yg

npc["year_group"] = npc["tp_gyear"].apply(get_year_group_10)

955553


In [24]:
r.columns

Index(['1975-85', '1985-95', '1995-05', '2005-15'], dtype='object')

In [50]:
c = "direct_cite"
m2 = npc
npc_T = m2.loc[(m2["inv_msa_match"] == True)].groupby("year_group")
npc_F = m2.loc[(m2["inv_msa_match"] == False)].groupby("year_group")

npc_T_mean = npc_T.mean()[c]
npc_F_mean = npc_F.mean()[c]
# tot_size = m2.groupby("year_group").size()
# npc_T_size = npc_T.size()/tot_size
# npc_F_size = npc_F.size()/tot_size

npc_T_size = npc_T.size().astype(int).astype(str)
npc_F_size = npc_F.size().astype(int).astype(str)

vals = [sp.stats.ttest_ind(npc_T.get_group(n)[c], npc_F.get_group(n)[c], nan_policy="omit")
       for n in npc_T.groups.keys()]
t = [i[0] for i in vals]
p = [i[1] for i in vals]

r = pd.DataFrame({"Pr Direct Cite, $I(MSA \,Match) = T$": np.round(npc_T_mean,3).tolist(),
                "Pr Direct Cite, $I(MSA \,Match) = F$": np.round(npc_F_mean,3).tolist(),
                "$N, I(MSA \,Match) = T$": npc_T_size,
                "$N, I(MSA \,Match) = F$": npc_F_size,  
                                  "$t$-value":np.round(t,2),
                                  "$p$-value":np.round(p,2),
                                 }, index=npc_T_mean.index.tolist())
r = r[["Pr Direct Cite, $I(MSA \,Match) = T$", "Pr Direct Cite, $I(MSA \,Match) = F$", "$N, I(MSA \,Match) = T$",
        "$N, I(MSA \,Match) = F$",  "$t$-value", "$p$-value"]]
dc_msa = r.T

c = "sim_docvecs"
npc_T_mean = npc_T.mean()[c]
npc_F_mean = npc_F.mean()[c]

vals = [sp.stats.ttest_ind(npc_T.get_group(n)[c], npc_F.get_group(n)[c], nan_policy="omit")
       for n in npc_T.groups.keys()]
t = [i[0] for i in vals]
p = [i[1] for i in vals]

r = pd.DataFrame({"Sim DV, $I(MSA \,Match) = T$": np.round(npc_T_mean,3).tolist(),
                "Sim DV, $I(MSA \,Match) = F$": np.round(npc_F_mean,3).tolist(),
                "$N, I(MSA \,Match) = T$": npc_T_size,
                "$N, I(MSA \,Match) = F$": npc_F_size,  
                                  "$t$-value":np.round(t,2),
                                  "$p$-value":np.round(p,2),
                                 }, index=npc_T_mean.index.tolist())

r = r[["Sim DV, $I(MSA \,Match) = T$", "Sim DV, $I(MSA \,Match) = F$", "$N, I(MSA \,Match) = T$",
        "$N, I(MSA \,Match) = F$",  "$t$-value", "$p$-value"]]
dv_msa = r.T

In [55]:
print(dc_msa.to_latex(index=True,escape=False,column_format="lcccc"))
print(dv_msa.to_latex(index=True,escape=False,column_format="lcccc"))

\begin{tabular}{lcccc}
\toprule
{} & 1975-85 & 1985-95 & 1995-05 & 2005-15 \\
\midrule
Pr Direct Cite, $I(MSA \,Match) = T$ &   0.061 &    0.06 &   0.023 &   0.093 \\
Pr Direct Cite, $I(MSA \,Match) = F$ &   0.018 &   0.024 &   0.006 &   0.023 \\
$N, I(MSA \,Match) = T$              &      33 &     990 &   19848 &  212732 \\
$N, I(MSA \,Match) = F$              &     168 &    1780 &   45258 &  521351 \\
$t$-value                            &    1.44 &    4.76 &   19.19 &  136.81 \\
$p$-value                            &    0.15 &       0 &       0 &       0 \\
\bottomrule
\end{tabular}

\begin{tabular}{lcccc}
\toprule
{} & 1975-85 & 1985-95 & 1995-05 & 2005-15 \\
\midrule
Sim DV, $I(MSA \,Match) = T$ &   0.537 &   0.393 &   0.356 &   0.331 \\
Sim DV, $I(MSA \,Match) = F$ &    0.23 &   0.212 &   0.378 &   0.522 \\
$N, I(MSA \,Match) = T$      &      33 &     990 &   19848 &  212732 \\
$N, I(MSA \,Match) = F$      &     168 &    1780 &   45258 &  521351 \\
$t$-value                    & 

In [20]:

npc_T = npc.groupby("year_group")
npc_F = mdc.groupby("year_group")

npc_T_size = npc_T.size().astype(int).astype(str)
npc_F_size = npc_F.size().astype(int).astype(str)

# Sim DocVecs
c = "op_cites_tp"
npc_T_mean = npc_T.mean()[c]
npc_F_mean = npc_F.mean()[c]

vals = [sp.stats.ttest_ind(npc_T.get_group(n)[c], npc_F.get_group(n)[c], nan_policy="omit")
       for n in npc_T.groups.keys()]
t = [i[0] for i in vals]
p = [i[1] for i in vals]

r = pd.DataFrame({"Sim DV, Common Non-Pat Cited": np.round(npc_T_mean,3).tolist(),
                "Sim DV, Common Pat Cited": np.round(npc_F_mean,3).tolist(),
                "$N$, Common Non-Pat Cited": npc_T_size,
                "$N$, Common Pat Cited": npc_F_size,  
                                  "$t$-value":np.round(t,2),
                                  "$p$-value":np.round(p,2),
                                 }, index=npc_T_mean.index.tolist())
r = r[["Sim DV, Common Non-Pat Cited", "Sim DV, Common Pat Cited", "$N$, Common Non-Pat Cited",
        "$N$, Common Pat Cited",  "$t$-value", "$p$-value"]]
dc_comp = r.T

# Direct cites
c = "sim_docvecs"
npc_T_mean = npc_T.mean()[c]
npc_F_mean = npc_F.mean()[c]

vals = [sp.stats.ttest_ind(npc_T.get_group(n)[c], npc_F.get_group(n)[c], nan_policy="omit")
       for n in npc_T.groups.keys()]
t = [i[0] for i in vals]
p = [i[1] for i in vals]

r = pd.DataFrame({"Pr Direct Cite, Common Non-Pat Cited": np.round(npc_T_mean,3).tolist(),
                "Pr Direct Cite, Common Pat Cited": np.round(npc_F_mean,3).tolist(),
                "$N$, Common Non-Pat Cited": npc_T_size,
                "$N$, Common Pat Cited": npc_F_size,  
                                  "$t$-value":np.round(t,2),
                                  "$p$-value":np.round(p,2),
                                 }, index=npc_T_mean.index.tolist())
r = r[["Pr Direct Cite, Common Non-Pat Cited", "Pr Direct Cite, Common Pat Cited", "$N$, Common Non-Pat Cited",
        "$N$, Common Pat Cited",  "$t$-value", "$p$-value"]]
dv_comp = r.T

In [21]:
print(dc_comp.to_latex(index=True,escape=False,column_format="lcccc"))
print(dv_comp.to_latex(index=True,escape=False,column_format="lcccc"))

\begin{tabular}{lcccc}
\toprule
{} & 1975-85 & 1985-95 & 1995-05 &  2005-15 \\
\midrule
Sim DV, Common Non-Pat Cited &   0.025 &   0.037 &   0.011 &    0.043 \\
Sim DV, Common Pat Cited     &   0.068 &   0.059 &   0.028 &    0.011 \\
$N$, Common Non-Pat Cited    &     201 &    2770 &   65106 &   734083 \\
$N$, Common Pat Cited        &   18954 &  149593 &  871983 &  1404639 \\
$t$-value                    &   -2.42 &   -4.96 &  -24.92 &   152.15 \\
$p$-value                    &    0.02 &       0 &       0 &        0 \\
\bottomrule
\end{tabular}

\begin{tabular}{lcccc}
\toprule
{} & 1975-85 & 1985-95 & 1995-05 &  2005-15 \\
\midrule
Pr Direct Cite, Common Non-Pat Cited &    0.28 &   0.277 &   0.371 &    0.467 \\
Pr Direct Cite, Common Pat Cited     &   0.272 &   0.255 &   0.225 &     0.22 \\
$N$, Common Non-Pat Cited            &     201 &    2770 &   65106 &   734083 \\
$N$, Common Pat Cited                &   18954 &  149593 &  871983 &  1404639 \\
$t$-value                          